In [ ]:
import numpy as np
from collections import OrderedDict
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
# Import the noise model object
from qiskit.providers.aer.noise import NoiseModel

# Import all standard errors
from qiskit.providers.aer.noise.errors import *  

# Import noise base clases
from qiskit.providers.aer.noise.errors.quantum_error import QuantumError
from qiskit.providers.aer.noise.errors.readout_error import ReadoutError
# import NoisyQuantumCircuit as NoisyQuantumCircuit

# Use PySCF, a classical computational chemistry software package, to compute the one- and
# two-body integrals in molecular-orbital basis, necessary to form the Fermionic operator
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.735', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()
num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2

# Build the qubit operator, which is the input to the VQE algorithm in Aqua
ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
map_type = 'PARITY'
qubitOp = ferOp.mapping(map_type)
qubitOp = qubitOp.two_qubit_reduced_operator(num_particles)
num_qubits = qubitOp.num_qubits

# set the backend for the quantum computation
from qiskit import Aer
# backend = Aer.get_backend('statevector_simulator')
backend = Aer.get_backend('qasm_simulator')

# setup a classical optimizer for VQE
from qiskit.aqua.components.optimizers import L_BFGS_B
optimizer = L_BFGS_B()

# setup the initial state for the variational form
from qiskit.chemistry.aqua_extensions.components.initial_states import HartreeFock
init_state = HartreeFock(num_qubits, num_spin_orbitals, num_particles)

# setup the variational form for VQE
from qiskit.aqua.components.variational_forms import RYRZ
var_form = RYRZ(num_qubits, initial_state=init_state)


T1Vec = np.linspace(30000,8000,5)
eigValVec = np.zeros((len(T1Vec),))
for idx in range(len(T1Vec)):
    # T1 = 12000  # nanoseconds
    T1 = T1Vec[idx]
    T2 = T1 * 2 # nanoseconds
    Tau0 = 50   # nanoseconds    
    pulseWidth = 1.25*Tau0

    errors1Qubit = thermal_relaxation_error(T1, T2, pulseWidth)
    errors2Qubit = thermal_relaxation_error(T1, T2, pulseWidth).kron(
                    thermal_relaxation_error(T1, T2, pulseWidth))


    # setup and run VQE
    from qiskit.aqua.algorithms import VQE
    algorithm = VQE(qubitOp, var_form, optimizer, [errors1Qubit],[errors2Qubit])
    # algorithm.print_setting
    # print(qubitOp,algorithm.solve())

    # algorithm.construct
    # print(algorithm)

    result = algorithm.run(backend)
    eigValVec[idx] = result['eigvals'][0]
    print(eigValVec[idx])

Qasm simulation does not work on matrix mode, changing the operator_mode to "paulis"
Qasm simulation does not work on matrix mode, changing the operator_mode to "paulis"


-1.0004696761474001


Qasm simulation does not work on matrix mode, changing the operator_mode to "paulis"


-0.865467857433377


In [ ]:
# print(result['eigvals'][0])

In [ ]:
# import numpy as np
# T1Vec = np.linspace(12000,500,4)
# print(T1Vec)